In [103]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

# import glob, os
# glob.glob(os.path.join(os.environ["SPARK_HOME"], "conf", "spark*"))
# print(os.environ)

In [112]:
#read csv

testdf = pd.read_csv('datasets/test.csv')
# traindf = pd.read_csv('datasets/train.csv')
alldf = pd.read_csv('datasets/submit.csv')

# print(testdf.shape, traindf.shape)

# print('cleaning dataframes')

# print(np.sum(testdf.isnull()))

testdf[['title', 'author']] = testdf[['title', 'author']].fillna(' ')

testdf = testdf.dropna()
# traindf = traindf.dropna()

# print(testdf.shape, traindf.shape)

ytestdf = testdf.drop(['title', 'author', 'text'], axis=1)

In [113]:
# ytestdf = ytestdf.merge(alldf, left_on='id', right_on='id', how='left')[['id', 'label']]
ytestdf = pd.merge(left = ytestdf, right=alldf, left_on='id', right_on='id', how='left')
print(ytestdf.shape)

(5193, 2)


In [114]:
spark = SparkSession.builder.\
    master('local[*]').\
        config('spark.driver.memory', '10g').\
            appName('fakenews').\
                getOrCreate()

print(spark.sparkContext.uiWebUrl)
print(spark.sparkContext.getConf().getAll())
    # master('local[*]').\
    #     config('spark.driver.memory','10g').\
print(spark)

http://Ryzen-PC:4040
[('spark.app.name', 'fakenews'), ('spark.driver.host', 'Ryzen-PC'), ('spark.driver.memory', '10g'), ('spark.executor.id', 'driver'), ('spark.sql.warehouse.dir', 'file:/C:/Users/rushi/Documents/GitHub/fake-news-detection/spark-warehouse'), ('spark.app.id', 'local-1648561256887'), ('spark.app.startTime', '1648561256002'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.port', '58487')]


In [115]:
from pyspark.sql.types import StructField, StructType
from pyspark.sql.types import IntegerType, StringType, DoubleType
from pyspark.sql import DataFrame
from pyspark.sql.functions import monotonically_increasing_id

trainschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType()),\
                StructField('label', IntegerType())]) 

testschema = StructType([StructField('id', IntegerType()),\
    StructField('title', StringType()),\
        StructField('author', StringType()),\
            StructField('text', StringType())]) 

trainset = spark.read.csv('datasets/train.csv', header=True, schema=trainschema)
testset = spark.read.csv('datasets/test.csv', header=True, schema=testschema)



trainset = trainset.fillna(' ', subset=['title', 'author']).dropna()
testset = testset.fillna(' ', subset=['title', 'author']).dropna()

print(trainset.count(), testset.count())

# trainset.printSchema()
# testset.printSchema()

12256 5194


In [116]:
trainset.show(10)

+---+--------------------+--------------------+--------------------+-----+
| id|               title|              author|                text|label|
+---+--------------------+--------------------+--------------------+-----+
|  1|FLYNN: Hillary Cl...|     Daniel J. Flynn|Ever get the feel...|    0|
|  5|Jackie Mason: Hol...|     Daniel Nussbaum|In these trying t...|    0|
|  7|Benoît Hamon Wins...|     Alissa J. Rubin|PARIS  —   France...|    0|
|  8|Excerpts From a D...|                 nan|Donald J. Trump i...|    0|
|  9|A Back-Channel Pl...|Megan Twohey and ...|A week before Mic...|    0|
| 10|Obama’s Organizin...|         Aaron Klein|Organizing for Ac...|    0|
| 11|"BBC Comedy Sketc...|     Chris Tomlinson|The BBC produced ...|    0|
| 15|In Major League S...|       Jack Williams|Guillermo Barros ...|    0|
| 16|Wells Fargo Chief...|Michael Corkery a...|The scandal engul...|    0|
| 19|Chuck Todd: ’Buzz...|           Jeff Poor|Wednesday after  ...|    0|
+---+--------------------

In [117]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF, CountVectorizer, StopWordsRemover, IDF
from pyspark.sql.functions import regexp_replace

regex = '[,\\-]'

trainset = trainset.withColumn('text', regexp_replace(trainset.text, regex, ' '))
testset = testset.withColumn('text', regexp_replace(testset.text, regex, ' '))

#tonkenize text

tokenizer = Tokenizer(inputCol='text', outputCol='words')
# wordsdata = tokenizer.transform(trainset)

stopwr = StopWordsRemover(inputCol='words', outputCol='filtered')
# wordsdata = stopwr.transform(wordsdata)

hashing = HashingTF(inputCol='filtered', outputCol='hashed', numFeatures=32)
# wordsdata = hashing.transform(wordsdata)

hashingIDF = IDF(inputCol='hashed', outputCol='features')
# model = hashingIDF.fit(wordsdata)
# wordsdata = model.transform(wordsdata)

# vector = CountVectorizer(inputCol='filtered', outputCol='featuresCV')
# model = vector.fit(wordsdata)

# wordsdata = model.transform(wordsdata)
#create a model
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(maxIter=1)
# gbt.fit(wordsdata)

# wordsdata.show(10)

In [118]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

pipeline = Pipeline(stages=[tokenizer, stopwr, hashing, hashingIDF, gbt])

pipeline = pipeline.fit(trainset.select(col('text'), col('label').cast('int')))


predictions = pipeline.transform(testset.select(col('text'))) 

In [119]:
from sklearn.metrics import accuracy_score, classification_report

Yspark_pred = predictions.toPandas()
Yspark_pred = Yspark_pred['prediction']

print(Yspark_pred.shape, ytestdf.shape)

FK_accuracy = accuracy_score(ytestdf, Yspark_pred) * 100
FK_classification_report = classification_report(ytestdf, Yspark_pred)

print('The accuracy of GBT Classifier to predict fake news is ', FK_accuracy)
print('Classification Report: \n', FK_classification_report)

(5194,) (5193, 2)


ValueError: Found input variables with inconsistent numbers of samples: [5193, 5194]

In [ ]:
spark.stop()